## 1. 라이브러리 불러오기

In [1]:
import sys
import glob
import cv2
import numpy as np

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils, datasets, models
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.optim import lr_scheduler

from torch.autograd import Variable

from matplotlib import pyplot as plt
from time import time

import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold

# import wandb

# from augraphy import *

In [2]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

## 2. Custom Dataset

In [3]:
class ImageDataset(Dataset):
    def __init__(self, csv, path, album_transform=None, augraphy_transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path 
        self.album_transform = album_transform
        self.augraphy_transform = augraphy_transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))                  

        
        if self.augraphy_transform:
            img = self.augraphy_transform(img)

        if self.album_transform:
            img = self.album_transform(image=img)['image']
        
        return img, target

## 3. Training Pipeline

In [4]:
def training(model, dataloader, device, criterion, optimizer, epoch, num_epochs):
    model.train()
    train_loss = 0.0
    preds_list = []
    targets_list = []

    tbar = tqdm(dataloader)
    for images, labels in tbar:
        images = images.type(torch.cuda.FloatTensor)
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(outputs.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(labels.detach().cpu().numpy())

        tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Train Loss : {loss.item():.4f}")

    train_loss = train_loss / (len(dataloader))
    train_acc = accuracy_score(preds_list, targets_list)
    train_f1 = f1_score(preds_list, targets_list, average='macro')

    metrics = {
        'train_loss' : train_loss,
        'train_acc' : train_acc,
        'train_f1' : train_f1
    }

    return model, metrics

def evaluation(model, dataloader, device, criterion, epoch, num_epochs):
    model.eval()
    valid_loss = 0.0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        tbar = tqdm(dataloader)
        for images, labels in tbar:
            images = images.type(torch.cuda.FloatTensor)
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()
            preds_list.extend(outputs.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(labels.detach().cpu().numpy())

            tbar.set_description(f"Epcoh [{epoch+1}/{num_epochs}] Valid Loss : {valid_loss:.4f}")

    valid_loss /= len(dataloader)
    valid_acc = accuracy_score(preds_list, targets_list)
    valid_f1 = f1_score(preds_list, targets_list, average='macro')

    metrics = {
        'valid_loss' : valid_loss,
        'valid_acc' : valid_acc,
        'valid_f1' : valid_f1
    }

    return model, metrics

def training_loop(model, train_dataloader, valid_dataloader, criterion, optimizer, device, num_epochs, model_path, model_name, patience):

    best_valid_loss = float('inf')
    valid_max_accuracy = -1
    valid_max_f1 = -1
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model, train_metrics = training(model, train_dataloader, device, criterion, optimizer, epoch, num_epochs)
        model, valid_metrics = evaluation(model, valid_dataloader, device, criterion, epoch, num_epochs)

        monitoring_value = {
            'train_loss' : train_metrics['train_loss'],
            'train_accuracy' : train_metrics['train_acc'],
            'train_f1' : train_metrics['train_f1'],
            'valid_loss' : valid_metrics['valid_loss'],
            'valid_accuracy' : valid_metrics['valid_acc'],
            'valid_f1' : valid_metrics['valid_f1']
        }

        if valid_max_accuracy < valid_metrics['valid_acc']:
            valid_max_accuracy = valid_metrics['valid_acc']
        
        if valid_max_f1 < valid_metrics['valid_f1']:
            valid_max_f1 = valid_metrics['valid_f1']
            torch.save(model.state_dict(), model_path+f"/model_{model_name}.pt")

        if best_valid_loss > valid_metrics['valid_loss']:
            best_valid_loss = valid_metrics['valid_loss']
            early_stop_counter = 0

        else:
            early_stop_counter += 1
            
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss : {train_metrics['train_loss']:.4f}, Train Acc : {train_metrics['train_acc']:.4f}, 'Train F1 : {train_metrics['train_f1']:.4f}, Valid Loss : {valid_metrics['valid_loss']:.4f}, Valid Acc : {valid_metrics['valid_acc']:.4f}, Valid F1 : {valid_metrics['valid_f1']}")

        if early_stop_counter >= patience:
            print('Early Stopping!')        
            break

    return model, valid_max_accuracy, valid_max_f1


## 4. Data Load

In [5]:
img_csv_path = '/data/ephemeral/home/data/aug_dataset/merged_df.csv'
df_img = pd.read_csv(img_csv_path)


In [6]:
df_img['target2'].value_counts()

target2
13    3256
7     3178
10    3162
2     3100
16    3100
6     3100
0     3100
12    3100
15    3100
5     3100
8     3100
4     3100
9     3100
14    3084
11    3038
3     3038
1     2852
Name: count, dtype: int64

In [7]:
img_csv_path = '/data/ephemeral/home/data/aug_dataset/merged_df.csv'
img_path = '/data/ephemeral/home/data/aug_dataset/aug_2'

submission_path = '/data/ephemeral/home/data/sample_submission.csv'
test_img_path = '/data/ephemeral/home/data/test'

totensor_transform = A.Compose([A.Resize(380, 380), ToTensorV2()])
test_transform = A.Compose([
    A.Resize(380, 380),
    ToTensorV2()
])

train_dataset = ImageDataset(img_csv_path, img_path, album_transform=totensor_transform, augraphy_transform=None)
test_dataset = ImageDataset(submission_path, test_img_path, album_transform=test_transform, augraphy_transform=None)

print(len(train_dataset), len(test_dataset))

52608 3140


In [8]:
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## 5. Train Model

### Hyper Parameter 정의

In [9]:
from sklearn.model_selection import KFold

In [10]:

# Define K-fold cross-validation parameters
n_splits = 5
kf = KFold(n_splits=n_splits)

# Initialize empty list to store models
models = []

# Define training parameters
lr = 0.001
epochs = 10
batch_size = 32

model_number = 1

# Perform K-fold cross-validation
for train_index, val_index in kf.split(train_dataset):
    train_sampler = torch.utils.data.SubsetRandomSampler(train_index)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_index)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
    valid_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler)

    model = timm.create_model('efficientnet_b4', pretrained=True)
    in_features = model.classifier.in_features
    classifier = nn.Sequential(
    nn.Linear(in_features, 1024),
    nn.BatchNorm1d(1024),
    nn.SiLU(),
    nn.Dropout(p=0.2),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.SiLU(),
    nn.Dropout(p=0.2),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.SiLU(),
    nn.Dropout(p=0.2),
    nn.Linear(256, 17),
    )

    model.classifier = classifier


    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 100
    batch_size=32
    model_path = '/data/ephemeral/home/models'

    model_name = f'effb4-{model_number}'

    model, valid_max_accuracy, valid_max_f1 = training_loop(model, 
                                                            train_dataloader, 
                                                            valid_dataloader, 
                                                            criterion, 
                                                            optimizer, 
                                                            device, 
                                                            num_epochs, 
                                                            model_path, 
                                                            model_name, 
                                                            10)
    
    print(f'{model_name} training finish... ')
    print('='*80)
    print()
    
    model_number += 1

    models.append(model)



Epcoh [1/100] Valid Loss : 57.2482: 100%|██████████| 329/329 [01:11<00:00,  4.60it/s]


Epoch [1/100], Train Loss : 0.3061, Train Acc : 0.8972, 'Train F1 : 0.8972, Valid Loss : 0.1740, Valid Acc : 0.9378, Valid F1 : 0.9323137264055719


Epcoh [2/100] Valid Loss : 48.0102: 100%|██████████| 329/329 [01:10<00:00,  4.63it/s]


Epoch [2/100], Train Loss : 0.0957, Train Acc : 0.9691, 'Train F1 : 0.9692, Valid Loss : 0.1459, Valid Acc : 0.9541, Valid F1 : 0.948976421628564


Epcoh [3/100] Valid Loss : 44.8089: 100%|██████████| 329/329 [01:10<00:00,  4.67it/s]


...

Epoch [34/100], Train Loss : 0.0093, Train Acc : 0.9976, 'Train F1 : 0.9976, Valid Loss : 0.0288, Valid Acc : 0.9925, Valid F1 : 0.9923108153465838


Epcoh [35/100] Valid Loss : 10.9991: 100%|██████████| 329/329 [01:38<00:00,  3.34it/s]


Epoch [35/100], Train Loss : 0.0082, Train Acc : 0.9977, 'Train F1 : 0.9977, Valid Loss : 0.0334, Valid Acc : 0.9928, Valid F1 : 0.9927432197010596


Epcoh [36/100] Valid Loss : 8.7833: 100%|██████████| 329/329 [01:38<00:00,  3.34it/s]

Epoch [36/100], Train Loss : 0.0088, Train Acc : 0.9975, 'Train F1 : 0.9975, Valid Loss : 0.0267, Valid Acc : 0.9935, Valid F1 : 0.9932363355854712
Early Stopping!
effb4-5 training finish... 



In [11]:
preds_lists = []
idx = 1

for model in models:
    preds_list = []
    for images, labels in tqdm(test_dataloader):
        images = images.type(torch.cuda.FloatTensor)
        images = images.to(device)

        with torch.no_grad():
            preds = model(images)
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    preds_lists.append(preds_list)
    
    print(f'model {idx} inference finish... ')
    print('='*80)
    print()
    idx += 1

100%|██████████| 99/99 [00:30<00:00,  3.29it/s]


model 1 inference finish... 



100%|██████████| 99/99 [00:29<00:00,  3.37it/s]


model 2 inference finish... 



100%|██████████| 99/99 [00:29<00:00,  3.36it/s]


model 3 inference finish... 



100%|██████████| 99/99 [00:29<00:00,  3.36it/s]


model 4 inference finish... 



100%|██████████| 99/99 [00:29<00:00,  3.36it/s]

model 5 inference finish... 



In [12]:
pred_df = pd.read_csv("/data/ephemeral/home/data/sample_submission.csv")

pred_columns = ['t1', 't2', 't3', 't4', 't5']
for i in range(5):
    pred_df[pred_columns[i]] = preds_lists[i]

In [13]:
pred_df.to_csv('/data/ephemeral/home/submission/ensemble_total.csv', index=False)